In [1]:
import logging
from naslib.defaults.trainer import Trainer
from naslib.optimizers import DARTSOptimizer
from naslib.search_spaces import DartsSearchSpace
from naslib.utils import utils, setup_logger, get_config_from_args, set_seed, log_args
from naslib.search_spaces.core.graph import Graph, EdgeData
from naslib.search_spaces.core import primitives as ops
from torch import nn
from fvcore.common.config import CfgNode
from copy import deepcopy
from IPython.display import clear_output
import torch
from naslib.search_spaces.core.primitives import AbstractPrimitive
from activation_sub_func.binary_func import Maximum, Minimum, Sub, Add, Mul, Div, SigMul, ExpBetaSub2, ExpBetaSubAbs, \
    BetaMix
from activation_sub_func.unary_func import Power, Sin, Cos, Abs_op, Sign, Beta, Beta_mul, Beta_add, Log, Exp, \
    Sinh, Cosh, \
    Tanh, Asinh, Acosh, Atan, Maximum0, Minimum0, Sigmoid, LogExp, Exp2, Erf, Sinc

device: cuda:0
device: cpu
device: cuda:0
device: cuda:0
device: cuda:0
device: cuda:0


In [19]:
config = utils.get_config_from_args(config_type='nas')
config.optimizer = 'darts'
config.search.batch_size = 8 # for TF pool, 32 for personal
config.search.learning_rate = 0.0001
utils.set_seed(config.seed)
clear_output(wait=True)
utils.log_args(config)

logger = setup_logger(config.save + '/log.log')
logger.setLevel(logging.INFO)

[07/08 10:12:41 nl.utils.utils]: dataset....................................cifar10
[07/08 10:12:41 nl.utils.utils]: seed.............................................0
[07/08 10:12:41 nl.utils.utils]: search_space...........................nasbench201
[07/08 10:12:41 nl.utils.utils]: out_dir........................................run
[07/08 10:12:41 nl.utils.utils]: optimizer....................................darts
[07/08 10:12:41 nl.utils.utils]: searchacq_fn_optimization: random_sampling
acq_fn_type: its
arch_learning_rate: 0.0003
arch_weight_decay: 0.001
batch_size: 8
checkpoint_freq: 1000
cutout: False
cutout_length: 16
cutout_prob: 1.0
data_size: 25000
debug_predictor: False
drop_path_prob: 0.0
encoding_type: adjacency_one_hot
epochs: 100
fidelity: -1
gpu: None
grad_clip: 5
k: 10
learning_rate: 0.0001
learning_rate_min: 0.001
max_mutations: 1
momentum: 0.9
num_arches_to_mutate: 2
num_candidates: 20
num_ensemble: 3
num_init: 10
output_weights: True
population_size: 30
predictor_ty

In [20]:
class SimpleSearchSpace(Graph):

    OPTIMIZER_SCOPE = [
        'a_stage_1',
        'a_stage_2'
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        stages = ['a_stage_1', 'a_stage_2']

        # cell definition
        activation_cell = Graph()
        activation_cell.name = 'activation_cell'
        activation_cell.add_node(1) # input node
        activation_cell.add_node(2) # intermediate node
        activation_cell.add_node(3) # output node
        activation_cell.add_edges_from([(1, 2, EdgeData())]) # mutable intermediate edge
        activation_cell.add_edges_from([(2, 3, EdgeData().finalize())]) # immutable output edge

        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1) # input node
        self.add_node(2) # intermediate node
        for i, scope in zip(range(3, 5), stages):
            self.add_node(i, subgraph=deepcopy(activation_cell).set_scope(scope).set_input([i-1])) # activation cell i
            self.nodes[i]['subgraph'].name = scope
        self.add_node(5) # output node
        self.add_edges_from([(i, i+1, EdgeData()) for i in range(1, 5)])
        self.edges[1, 2].set('op',
            ops.Sequential(
                nn.Conv2d(3, 6, 5),
                nn.MaxPool2d(2),
                nn.Conv2d(6, 16, 5),
                nn.MaxPool2d(2),
                nn.Flatten()
            )) # convolutional edge
        self.edges[4, 5].set('op', 
            ops.Sequential(
                nn.Linear(400, 10), 
                nn.Softmax(dim=1)
            )) # linear edge
        
        for scope in stages:
            self.update_edges(
                update_func=lambda edge: self._set_ops(edge),
                scope=scope,
                private_edge_data=True,
            )

    def _set_ops(self, edge):
        edge.data.set('op', [
            ops.Sequential(nn.ReLU()),
            ops.Sequential(nn.Hardswish()),
            ops.Sequential(nn.LeakyReLU()),
            ops.Sequential(nn.Identity())
        ])

In [21]:
class stack():
    def __init__(self):
        pass
    def __call__(self, tensors, edges_data=None):
        return torch.stack(tensors)


class ComplexSearchSpace(Graph):

    OPTIMIZER_SCOPE = [
        'a_stage_1',
        'u_stage_1',
        'u_stage_2',
        'b_stage_1'
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        u_stages = ['u_stage_1', 'u_stage_2']
        
        # unary cell definition
        unary_cell = Graph()
        unary_cell.name = 'u_cell'
        unary_cell.add_node(1) # input node
        unary_cell.add_node(2) # intermediate node
        unary_cell.add_node(3) # output node
        unary_cell.add_edges_from([(1, 2, EdgeData())]) # mutable edge
        unary_cell.edges[1, 2].set('cell_name', 'u_cell')
        unary_cell.add_edges_from([(2, 3, EdgeData().finalize())]) # immutable edge
        
        # binary cell definition
        binary_cell = Graph()
        binary_cell.name = 'b_cell'
        binary_cell.add_node(1) # input node
        binary_cell.add_node(2) # input node
        binary_cell.add_node(3) # concatination node
        binary_cell.nodes[3]['comb_op'] = stack()
        binary_cell.add_node(4) # intermediate node
        binary_cell.add_node(5) # output node
        binary_cell.add_edges_from([(3, 4, EdgeData())]) # mutable edge
        binary_cell.edges[3, 4].set('cell_name', 'b_cell') 
        binary_cell.add_edges_from([(1, 3, EdgeData().finalize()),
                                    (2, 3, EdgeData().finalize()),
                                    (4, 5, EdgeData().finalize())]) # immutable edges
        
        # activation cell definition
        activation_cell = Graph()
        activation_cell.name = 'a_cell'
        activation_cell.add_node(1) # input node
        activation_cell.add_node(2, subgraph=deepcopy(unary_cell).set_scope('u_stage_1').set_input([1])) # unary node
        activation_cell.nodes[2]['subgraph'].name = 'u_stage_1'
        activation_cell.add_node(3, subgraph=deepcopy(unary_cell).set_scope('u_stage_2').set_input([1])) # unary node
        activation_cell.nodes[3]['subgraph'].name = 'u_stage_2'
        activation_cell.add_node(4, subgraph=deepcopy(binary_cell).set_scope('b_stage_1').set_input([2, 3])) # binary node
        activation_cell.nodes[4]['subgraph'].name = 'b_stage_1'
        activation_cell.add_node(5) # output node
        activation_cell.add_edges_from([(1, 2, EdgeData().finalize()), 
                                        (1, 3, EdgeData().finalize()),
                                        (2, 4, EdgeData().finalize()),
                                        (3, 4, EdgeData().finalize()), 
                                        (4, 5, EdgeData().finalize())])
        
        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1) # input node
        self.add_node(2) # intermediate node
        self.add_node(3, subgraph=deepcopy(activation_cell).set_input([2])) # activation cell
        self.nodes[3]['subgraph'].name = 'a_stage_1'
        self.add_node(4) # output node
        self.add_edges_from([(i, i+1, EdgeData()) for i in range(1, 4)])
        self.edges[1, 2].set('op',
            ops.Sequential(
                nn.Conv2d(3, 6, 5),
                nn.MaxPool2d(2),
                nn.Conv2d(6, 16, 5),
                nn.MaxPool2d(2),
                nn.Flatten()
            )) # convolutional edge
        self.edges[3, 4].set('op', 
            ops.Sequential(
                nn.Linear(400, 10), 
                nn.Softmax(dim=1)
            )) # linear edge
        
        for scope in u_stages:
            self.update_edges(
                update_func=lambda edge: self._set_unary_ops(edge),
                scope=scope,
                private_edge_data=True,
            ) # set unary cell ops
        
        self.update_edges(
            update_func=lambda edge: self._set_binary_ops(edge),
            scope='b_stage_1',
            private_edge_data=True
        ) # set binary cell ops
        

    def _set_unary_ops(self, edge):
        edge.data.set('op', [ops.Identity(), ops.Zero(stride=1)]) 
        
        
    def _set_binary_ops(self, edge):
        edge.data.set('op', [Minimum(), Maximum()]) 

In [22]:
class RNNSearchSpace(Graph):

    OPTIMIZER_SCOPE = [
        'a_stage_1',
        'u_stage_1',
        'u_stage_2',
        'u_stage_3',
        'u_stage_4',
        'b_stage_1',
        'b_stage_2'
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        u_stages = ['u_stage_1', 'u_stage_2', 'u_stage_3', 'u_stage_4']
        b_stages = ['b_stage_1', 'b_stage_2']
        
        # unary cell definition
        unary_cell = Graph()
        unary_cell.name = 'u_cell'
        unary_cell.add_node(1) # input node
        unary_cell.add_node(2) # intermediate node
        unary_cell.add_node(3) # output node
        unary_cell.add_edges_from([(1, 2, EdgeData())]) # mutable edge
        unary_cell.edges[1, 2].set('cell_name', 'u_cell')
        unary_cell.add_edges_from([(2, 3, EdgeData().finalize())]) # immutable edge
        
        # binary cell definition
        binary_cell = Graph()
        binary_cell.name = 'b_cell'
        binary_cell.add_node(1) # input node
        binary_cell.add_node(2) # input node
        binary_cell.add_node(3) # concatination node
        binary_cell.nodes[3]['comb_op'] = stack()
        binary_cell.add_node(4) # intermediate node
        binary_cell.add_node(5) # output node
        binary_cell.add_edges_from([(3, 4, EdgeData())]) # mutable edge
        binary_cell.edges[3, 4].set('cell_name', 'b_cell') 
        binary_cell.add_edges_from([(1, 3, EdgeData().finalize()),
                                    (2, 3, EdgeData().finalize()),
                                    (4, 5, EdgeData().finalize())]) # immutable edges
        
        # activation cell definition
        activation_cell = Graph()
        activation_cell.name = 'a_cell'
        activation_cell.add_node(1) # input node
        activation_cell.add_node(2, subgraph=deepcopy(unary_cell).set_scope('u_stage_1').set_input([1])) # unary cell 1
        activation_cell.nodes[2]['subgraph'].name = 'u_stage_1'
        activation_cell.add_node(3, subgraph=deepcopy(unary_cell).set_scope('u_stage_2').set_input([1])) # unary cell 2
        activation_cell.nodes[3]['subgraph'].name = 'u_stage_2'
        activation_cell.add_node(4, subgraph=deepcopy(unary_cell).set_scope('u_stage_3').set_input([1])) # unary cell 3
        activation_cell.nodes[4]['subgraph'].name = 'u_stage_3'
        activation_cell.add_node(5, subgraph=deepcopy(binary_cell).set_scope('b_stage_1').set_input([2, 3])) # binary cell 1
        activation_cell.nodes[5]['subgraph'].name = 'b_stage_1'
        activation_cell.add_node(6, subgraph=deepcopy(unary_cell).set_scope('u_stage_4').set_input([5])) # unary cell 4
        activation_cell.nodes[6]['subgraph'].name = 'u_stage_4'
        activation_cell.add_node(7, subgraph=deepcopy(binary_cell).set_scope('b_stage_2').set_input([4, 6])) # binary cell 2
        activation_cell.nodes[7]['subgraph'].name = 'b_stage_2'
        activation_cell.add_node(8) # output node
        activation_cell.add_edges_from([(1, 2, EdgeData().finalize()), 
                                        (1, 3, EdgeData().finalize()),
                                        (1, 4, EdgeData().finalize()),
                                        (2, 5, EdgeData().finalize()),
                                        (3, 5, EdgeData().finalize()), 
                                        (4, 7, EdgeData().finalize()),
                                        (5, 6, EdgeData().finalize()),
                                        (6, 7, EdgeData().finalize()),
                                        (7, 8, EdgeData().finalize())])
        
        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1) # input node
        self.add_node(2) # intermediate node
        self.add_node(3, subgraph=deepcopy(activation_cell).set_input([2])) # activation cell
        self.add_node(4) # output node
        self.add_edges_from([(i, i+1, EdgeData()) for i in range(1, 4)])
        self.edges[1, 2].set('op',
            ops.Sequential(
                nn.Conv2d(3, 6, 5),
                nn.MaxPool2d(2),
                nn.Conv2d(6, 16, 5),
                nn.MaxPool2d(2),
                nn.Flatten()
            )) # convolutional edge
        self.edges[3, 4].set('op', 
            ops.Sequential(
                nn.Linear(400, 10), 
                nn.Softmax(dim=1)
            )) # linear edge
        
        for scope in u_stages:
            self.update_edges(
                update_func=lambda edge: self._set_unary_ops(edge),
                scope=scope,
                private_edge_data=True,
            ) # set unary cell ops
        
        for scope in b_stages:
            self.update_edges(
                update_func=lambda edge: self._set_binary_ops(edge),
                scope=scope,
                private_edge_data=True
            ) # set binary cell ops
        

    def _set_unary_ops(self, edge, channels=None):
        edge.data.set('op', [
            ops.Identity(), 
            ops.Zero(stride=1)
        ]) 
        
        
    def _set_binary_ops(self, edge, channels=None):
        edge.data.set('op', [
            Minimum(),
            Maximum(),
        ]) 

In [23]:
class Stack(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())

    def forward(self, x, edge_data=None):
        return [x[0], x[1]]

    def get_embedded_ops(self):
        return None


class UnStack(AbstractPrimitive):
    def __init__(self, dim=1):
        super().__init__(locals())
        self.dim = dim

    def forward(self, x, edge_data=None):
        return x[self.dim]

    def get_embedded_ops(self):
        return None


class RNNResNet20SearchSpace(Graph):
    """
    https://www.researchgate.net/figure/ResNet-20-architecture_fig3_351046093
    """

    OPTIMIZER_SCOPE = [
        f"activation_{i}" for i in range(1, 20)
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        # cell definition
        activation_cell = Graph()
        activation_cell.name = 'activation_cell'
        activation_cell.add_node(1)  # input node
        activation_cell.add_node(2)  # unary node / intermediate node
        activation_cell.add_node(3)  # unary node / intermediate node
        activation_cell.add_node(4)  # binary node / output node
        activation_cell.add_edges_from([(1, 2, EdgeData())])  # mutable intermediate edge
        activation_cell.add_edges_from([(1, 3, EdgeData())])  # mutable intermediate edge

        activation_cell.add_edges_from([(2, 4, EdgeData().finalize())])  # mutable intermediate edge
        activation_cell.add_edges_from([(3, 4, EdgeData().finalize())])  # mutable intermediate edge
        activation_cell.nodes[4]['comb_op'] = Stack()

        activation_cell.add_node(5)  # binary node / output node
        activation_cell.add_edges_from([(4, 5, EdgeData())])  # mutable intermediate edge

        activation_cell.add_node(6)
        activation_cell.add_edges_from([(5, 6, EdgeData().finalize())])  # unary node / intermediate node
        activation_cell.add_node(7)
        activation_cell.add_edges_from([(6, 7, EdgeData())])  # mutable intermediate edge
        activation_cell.add_node(8)
        activation_cell.add_edges_from([(1, 8, EdgeData())])  # mutable intermediate edge

        activation_cell.add_node(9)
        activation_cell.add_edges_from([(8, 9, EdgeData().finalize())])  # mutable intermediate edge
        activation_cell.add_edges_from([(7, 9, EdgeData().finalize())])  # mutable intermediate edge
        activation_cell.nodes[9]['comb_op'] = Stack()

        activation_cell.add_node(10)
        activation_cell.add_edges_from([(9, 10, EdgeData())])  # mutable intermediate edge

        activation_cell.add_node(11)
        activation_cell.add_edges_from([(10, 11, EdgeData().finalize())])  # mutable intermediate edge

        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1)  # input
        self.add_node(2)  # intermediate
        self.add_node(3,
                      subgraph=activation_cell.copy().set_scope("activation_1").set_input([2]))  # activation cell 3
        self.nodes[3]['subgraph'].name = "activation_1"
        self.update_edges(
            update_func=lambda edge: self._set_ops(edge, 16),
            scope=f"activation_{1}",
            private_edge_data=True, )

        self.add_node(4)
        self.add_node(5,
                      subgraph=activation_cell.copy().set_scope("activation_2").set_input([4]))  # activation cell 3
        self.nodes[5]['subgraph'].name = "activation_2"
        self.update_edges(
            update_func=lambda edge: self._set_ops(edge, 16),
            scope=f"activation_{2}",
            private_edge_data=True, )

        self.add_node(6)
        # Todo add option here with a func which has a arg channels
        self.add_node(7,
                      subgraph=activation_cell.copy().set_scope("activation_3").set_input([6]))  # activation cell 3
        self.nodes[7]['subgraph'].name = "activation_3"
        self.update_edges(
            update_func=lambda edge: self._set_ops(edge, 16),
            scope=f"activation_{3}",
            private_edge_data=True, )

        self.add_edges_from([
            (1, 2, EdgeData()),
            (2, 3, EdgeData()),
            (3, 4, EdgeData()),
            (4, 5, EdgeData()),
            (5, 6, EdgeData()),
            (3, 6, EdgeData()),
            (6, 7, EdgeData())
        ])

        self.edges[1, 2].set('op',
                             ops.Sequential(nn.Conv2d(3, 16, 3, padding=1), ))  # convolutional edge
        self.edges[3, 4].set('op',
                             ops.Sequential(nn.Conv2d(16, 16, 3, padding=1), ))  # convolutional edge
        self.edges[5, 6].set('op',
                             ops.Sequential(nn.Conv2d(16, 16, 3, padding=1), ))  # convolutional edge

        conv_option = {
            "in_channels": 16,
            "out_channels": 16,
            "kernel_size": 3,
            "padding": 1
        }
        self._create_base_block(7, 4, activation_cell, conv_option)
        self._create_base_block(11, 6, activation_cell, conv_option)

        conv_option_a = {
            "in_channels": 16,
            "out_channels": 32,
            "kernel_size": 3,
            "padding": 1,
            "stride": 2
        }
        conv_option_b = {
            "in_channels": 16,
            "out_channels": 32,
            "kernel_size": 1,
            "padding": 0,
            "stride": 2
        }
        self._create_reduction_block(15, 8, activation_cell, conv_option_a, conv_option_b)

        conv_option = {
            "in_channels": 32,
            "out_channels": 32,
            "kernel_size": 3,
            "padding": 1
        }
        self._create_base_block(19, 10, activation_cell, conv_option)
        self._create_base_block(23, 12, activation_cell, conv_option)

        conv_option_a = {
            "in_channels": 32,
            "out_channels": 64,
            "kernel_size": 3,
            "padding": 1,
            "stride": 2
        }
        conv_option_b = {
            "in_channels": 32,
            "out_channels": 64,
            "kernel_size": 1,
            "padding": 0,
            "stride": 2
        }
        self._create_reduction_block(27, 14, activation_cell, conv_option_a, conv_option_b)

        conv_option = {
            "in_channels": 64,
            "out_channels": 64,
            "kernel_size": 3,
            "padding": 1
        }
        self._create_base_block(31, 16, activation_cell, conv_option)
        self._create_base_block(34, 18, activation_cell, conv_option)

        # add head
        self.add_node(39)
        self.add_edges_from([
            (38, 39, EdgeData())
        ])
        self.edges[38, 39].set('op',
                               ops.Sequential(
                                   nn.AvgPool2d(8),
                                   nn.Flatten(),
                                   nn.Linear(64, 10),
                                   nn.Softmax()
                               ))  # convolutional edge
        self.add_node(40)
        self.add_edges_from([
            (39, 40, EdgeData().finalize())
        ])

    def _create_base_block(self, start: int, stage: int, cell, conv_option: dict):
        self.add_node(start + 1)

        self.add_node(start + 2, subgraph=cell.copy().set_scope(f"activation_{stage}").set_input(
            [start + 1]))  # activation cell 3
        self.nodes[start + 2]['subgraph'].name = f"activation_{stage}"
        self.update_edges(
            update_func=lambda edge: self._set_ops(edge, conv_option["out_channels"]),
            scope=f"activation_{stage}",
            private_edge_data=True, )

        self.add_node(start + 3)

        self.add_node(start + 4, subgraph=cell.copy().set_scope(f"activation_{stage + 1}").set_input(
            [start + 3]))  # activation cell 3
        self.nodes[start + 4]['subgraph'].name = f"activation_{stage + 1}"
        self.update_edges(
            update_func=lambda edge: self._set_ops(edge, conv_option["out_channels"]),
            scope=f"activation_{stage + 1}",
            private_edge_data=True, )

        self.add_edges_from([
            (start, start + 1, EdgeData()),
            (start, start + 3, EdgeData()),
            (start + 1, start + 2, EdgeData()),
            (start + 2, start + 3, EdgeData()),
            (start + 3, start + 4, EdgeData()),
        ])

        self.edges[start, start + 1].set('op',
                                         ops.Sequential(nn.Conv2d(**conv_option), ))  # convolutional edge
        self.edges[start + 2, start + 3].set('op',
                                             ops.Sequential(nn.Conv2d(**conv_option), ))  # convolutional edge

    def _create_reduction_block(self, start: int, stage: int, cell, conv_option_a: dict, conv_option_b: dict):
        self.add_node(start + 1)

        self.add_node(start + 2, subgraph=cell.copy().set_scope(f"activation_{stage}").set_input(
            [start + 1]))  # activation cell 3
        self.nodes[start + 2]['subgraph'].name = f"activation_{stage}"
        self.update_edges(
            update_func=lambda edge: self._set_ops(edge, conv_option_a["out_channels"]),
            scope=f"activation_{stage}",
            private_edge_data=True, )

        self.add_node(start + 3)

        self.add_node(start + 4, subgraph=cell.copy().set_scope(f"activation_{stage + 1}").set_input(
            [start + 3]))  # activation cell 3
        self.nodes[start + 4]['subgraph'].name = f"activation_{stage + 1}"
        self.update_edges(
            update_func=lambda edge: self._set_ops(edge, conv_option_b["out_channels"]),
            scope=f"activation_{stage + 1}",
            private_edge_data=True, )

        self.add_edges_from([
            (start, start + 1, EdgeData()),
            (start, start + 3, EdgeData()),  # add conv
            (start + 1, start + 2, EdgeData()),
            (start + 2, start + 3, EdgeData()),
            (start + 3, start + 4, EdgeData()),
        ])

        self.edges[start, start + 1].set('op',
                                         ops.Sequential(nn.Conv2d(**conv_option_a), ))  # convolutional edge
        conv_option_a["in_channels"] = conv_option_a["out_channels"]
        conv_option_a["stride"] = 1

        self.edges[start, start + 3].set('op',
                                         ops.Sequential(nn.Conv2d(**conv_option_b), ))  # convolutional edge
        self.edges[start + 2, start + 3].set('op',
                                             ops.Sequential(nn.Conv2d(**conv_option_a), ))  # convolutional edge

    def _set_ops(self, edge, channels=32):
        # unary (1, 2), (1, 3), (1, 8), (6, 7)
        if (edge.head, edge.tail) in {(1, 2), (1, 3), (1, 8), (6, 7)}:
            edge.data.set("op", [
                ops.Identity(),
                ops.Zero(stride=1),
                Power(2),
                Power(3),
                Power(.5),
                Sin(),
                Cos(),
                Abs_op(),
                Sign(),
                Beta_mul(channels=channels),
                Beta_add(channels=channels),
                Log(),
                Exp(),
                Sinh(),
                Cosh(),
                Tanh(),
                Asinh(),
                Acosh(),
                Atan(),
                Sinc(),
                Maximum0(),
                Minimum0(),
                Sigmoid(),
                LogExp(),
                Exp2(),
                Erf(),
                Beta(channels=channels),
            ])
        # binary (4, 5), (9, 10)
        elif (edge.head, edge.tail) in {(4, 5), (9, 10)}:
            edge.data.set("op", [
                Add(),
                Sub(),
                Mul(),
                Div(),
                Maximum(),
                Minimum(),
                SigMul(),
                ExpBetaSub2(channels=channels),
                ExpBetaSubAbs(channels=channels),
                BetaMix(channels=channels),
            ])

In [24]:
search_space = RNNResNet20SearchSpace()

In [25]:
optimizer = DARTSOptimizer(config)
optimizer.adapt_search_space(search_space)

[07/08 10:12:44 nl.optimizers.oneshot.darts.optimizer]: Parsed graph:
Graph activation_1:
 Graph(
  (activation_1-edge(1,2)): MixedOp(
    (primitive-0): Identity()
    (primitive-1): Zero (stride=1)
    (primitive-2): Power()
    (primitive-3): Power()
    (primitive-4): Power()
    (primitive-5): Sin()
    (primitive-6): Cos()
    (primitive-7): Abs_op()
    (primitive-8): Sign()
    (primitive-9): Beta_mul()
    (primitive-10): Beta_add()
    (primitive-11): Log()
    (primitive-12): Exp()
    (primitive-13): Sinh()
    (primitive-14): Cosh()
    (primitive-15): Tanh()
    (primitive-16): Asinh()
    (primitive-17): Acosh()
    (primitive-18): Atan()
    (primitive-19): Sinc()
    (primitive-20): Maximum0()
    (primitive-21): Minimum0()
    (primitive-22): Sigmoid()
    (primitive-23): LogExp()
    (primitive-24): Exp2()
    (primitive-25): Erf()
    (primitive-26): Beta()
  )
  (activation_1-edge(1,3)): MixedOp(
    (primitive-0): Identity()
    (primitive-1): Zero (stride=1)
    

In [26]:
trainer = Trainer(optimizer, config)
trainer.search()

[07/08 10:12:45 nl.defaults.trainer]: param size = 0.284074MB
[07/08 10:12:45 nl.defaults.trainer]: Start training
Files already downloaded and verified
Files already downloaded and verified
[07/08 10:12:47 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
+0.002284, +0.000441, -0.001046, +0.001251, -0.000081, +0.000476, -0.000980, +0.001986, -0.000500, +0.000184, -0.000225, -0.001865, -0.000414, -0.000478, -0.001311, +0.001136, +0.000903, -0.001187, +0.000736, +0.000373, +0.001243, -0.000004, -0.002105, -0.000697, +0.000208, -0.000237, +0.001237, 0
+0.000643, -0.001894, +0.001188, +0.000884, +0.001262, +0.000749, -0.000507, +0.000371, -0.000850, +0.000645, +0.000021, +0.001923, +0.000255, -0.000010, +0.000037, -0.000944, +0.000469, -0.002251, -0.000803, +0.001068, +0.000936, +0.000429, -0.000888, +0.000341, -0.000459, +0.000400, +0.001379, 11
-0.001278, -0.000717, -0.001318, +0.001980, +0.000379, -0.000383, +0.001180, -0.000823, -0.001172, -0.002026, 

[07/08 10:12:48 nl.defaults.trainer]: Epoch 0-0, Train loss: nan, validation loss: nan, learning rate: [0.0001]
[07/08 10:12:53 nl.defaults.trainer]: Epoch 0-7, Train loss: nan, validation loss: nan, learning rate: [0.0001]


KeyboardInterrupt: 

In [53]:
trainer.evaluate_oneshot()

[07/08 10:08:45 nl.defaults.trainer]: Start one-shot evaluation
Files already downloaded and verified
Files already downloaded and verified


RuntimeError: CUDA error: CUBLAS_STATUS_ALLOC_FAILED when calling `cublasCreate(handle)`